In [130]:
# Import Basic Python Functionality
import numpy as np
import pandas as pd
import difflib

In [131]:
#Necessary Functions
def time_str_to_secs(time_str): #changes time from "MM:SS" to it's value in seconds (starting at 00:00)
    tmp_t=np.array(time_str.split(':')).astype('int')
    return tmp_t[0]*60+tmp_t[1]

def isnumber(st):
    try:
        int(st.split('-')[0])
        return True
    except:
        return False

def isword(st): #returns true if a string is all alphabetic characters
    alphabet=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','z','y']
    c,ll=0,len(st)
    for alph in st.lower():
        if alph in alphabet:
            c+=1
    return c==ll

def strs_to_labels(str_arr,ci=0,first_word=False): #takes in list of strings and outputs the first sequence of each that is not a word
    labels=[]
    for st in str_arr: #now loop through the "label" strings and pull out the first thing in that string that is not a word
        st_sp=st.split(' ') #break up the string by spaces
        c,alph=ci,'a' #ci is initial c, if we want to start at a different word in a string set ci higher, alph is just some alphanumeric word and isword will check if it is a word
        while isword(alph): #loop through all the segments of a string delimited by spaces and pull out the first thing that's not 100% alphabetic
            if c>=len(st_sp):
                alph='_'
            else:
                alph=st_sp[c]
            c+=1
        if isnumber(alph[0]): #If our str looks like this Boeing 777 we want it to become B777 not just 777 so we add this part in
            ind=c-2
            if first_word: ind=0
            alph=st_sp[ind][0]+alph
        labels.append(alph)
    return labels

def relabeler(labels,pre_st='',to_remove=['F','N','neo','LR']): #to get rid of things like A330neo since it is mostly the same as the A330
    labels_out=[]
    for lb in labels:
        tmp=lb
        if pre_st in lb[:1]:
            for si in to_remove:
                if si in lb:
                    tmp=lb.split(si)[0]
        labels_out.append(tmp)
    return labels_out

def label_replacer(labels,replace_dict={'A530':'A330','A580':'A380','A550':'A350','AZ30':'A330','AZ21':'A321'}): #because sometimes the screen recorder messes up on predictable characters
    for i in range(len(labels)):
        if labels[i] in list(replace_dict):
            labels[i]=replace_dict[labels[i]]
    return labels

def full_relabel(labels): #relabel several times based on first character, i.e. relabel just boeing so that we remove the F from B737F but not have it affect Airbus A320F for example
    return label_replacer(relabeler(relabeler(labels,pre_st='A',to_remove=['F','neo','-']),pre_st='E',to_remove=['-','LR']))

def weight_extractor(labels,dicti):
    weights,weight_classes=[],[]
    for l in labels:
        tmp_i=np.where(np.array(weight_labels)==dicti[l])[0][0]
        weights.append(weight_raw_labels[:,1][tmp_i])
        weight_classes.append(weight_raw_labels[:,-1][tmp_i])
    return weights,weight_classes

def generate_rough_dict(unique_a_labels,unique_w_labels,delim=',\n'):
    for a in unique_a_labels:
        mt=difflib.get_close_matches(a,unique_w_labels,n=3)
        if len(mt)==0: 
            print('"'+a+'":"'+''+'"',end=delim) 
        elif a==mt[0]:
            print('"'+a+'":"'+mt[0]+'"',end=delim) 
        else:
            mt=difflib.get_close_matches(a.split('-')[0],unique_w_labels,n=3)
            if len(mt)==0: 
                print('"'+a+'":"'+''+'"',end=delim) 
            else:
                print('"'+a+'":"'+mt[0]+'"',end=delim)

In [132]:
#These are the datafile lengths
[l_houston,l_berlin,l_amsterdam] = [3675, 3231, 7199]

#import label data
houston_times_labels=pd.read_csv('HoustonLabels.csv')[['start','stop','plane']].to_numpy()
berlin_times_labels=pd.read_csv('BerlinLabels.csv').to_numpy() #Times, Str, Aircraft
amsterdam_times_labels=pd.read_csv('AmsterdamLabels.csv')[['Raw Times(s)','Aircraft']].to_numpy()

#format times
houston_times=[[time_str_to_secs(time) for time in times] for times in houston_times_labels[:,:2]]
berlin_times=berlin_times_labels[:,0]
berlin_times=[time_str_to_secs(tm) for tm in berlin_times]+[l_berlin] #reformat into samples (also have to add the final time
berlin_times=[[berlin_times[i],berlin_times[i+1]] for i in range(len(berlin_times)-1)] #reformat so the time is a pair with the end of one tag at the start of the next tag
amsterdam_times=amsterdam_times_labels[:,0]
amsterdam_times=[[amsterdam_times[i],amsterdam_times[i+1]] for i in range(len(amsterdam_times)-1)]+[[amsterdam_times[-1],l_amsterdam]]

#format actual labels
houston_labels=houston_times_labels[:,2]
berlin_labels=strs_to_labels(berlin_times_labels[:,1],ci=2)
#berlin_labels=[label.split('-')[0] for label in berlin_labels]
amsterdam_labels=strs_to_labels(amsterdam_times_labels[:,1])
amsterdam_labels=[label.split('(')[0] for label in amsterdam_labels] #here I make the decision to throw away the additional identifiers, i.e. A330-200 -> A330
houston_labels,berlin_labels,amsterdam_labels=full_relabel(houston_labels),full_relabel(berlin_labels),full_relabel(amsterdam_labels)

#Import Weights Data
weight_raw_labels=pd.read_csv('aircraft_weight_classes.csv')[['type-[]','MTOW [kg]','ICAO category','FAA category']].to_numpy()
weight_labels=strs_to_labels(weight_raw_labels[:,0],first_word=True)
weight_labels=full_relabel(weight_labels)

h_to_w_dict={"A220":"A220","A300":"A300","A319":"A319","A320":"A320","A320N":"A320","A321":"A321","A350":"A350","A380":"A380","B737":"B737-300","B747F":"B747-8F","B767":"B767-300","B767F":"B767-300","B777":"B777F","B777F":"B777F","CRJ1000":"CRJ-1000","CRJ200":"CRJ-200","CRJ900":"CRJ-900","E145":"E145","E175":"E175","E190":"E190","MD80":"McDonnell-Douglas","tiny":"E100"}
b_to_w_dict={"A318":"A318","A319":"A319","A320":"A320","A321":"A321","A330":"A330","B737-800":"B737-800","B737-900":"B737-900","B767-300ER":"B767-300ER","B787-8":"B787-8","E190":"E190","E195":"E195"}
a_to_w_dict={"A300":"A300","A310":"A310","A318":"A318","A319":"A319","A320":"A320","A321":"A321","A330":"A330","A350":"A350","A380":"A380","B737":"B737-300","B737-700":"B737-700","B737-800":"B737-800","B747-400F":"B747-400","B747-8F":"B747-8F","B767-300ER":"B767-300ER","B767-300F":"B767-300","B777-200ER":"B777-200ER","B777-300ER":"B777-300ER","B777F":"B777F","B787-10":"B787-10","B787-8":"B787-8","B787-9":"B787-9","CRJ-1000":"CRJ-1000","CRJ-1000EL":"CRJ-1000EL","CRJ-900ER":"CRJ-900","CRJ-900LR":"CRJ-900","Dash-8-Q400":"Dash-8-Q400","E175":"E175","E190":"E190","E195":"E195"}

houston_weights,houston_weight_classes=weight_extractor(houston_labels,h_to_w_dict)
berlin_weights,berlin_weight_classes=weight_extractor(berlin_labels,b_to_w_dict)
amsterdam_weights,amsterdam_weight_classes=weight_extractor(amsterdam_labels,a_to_w_dict)

to_df = lambda labels,times,weights,w_classes:pd.DataFrame({"labels":labels,"start":np.array(times)[:,0],"stop":np.array(times)[:,1],"weight":weights,"w_classes":w_classes})

houston_df=to_df(houston_labels,houston_times,houston_weights,houston_weight_classes)
berlin_df=to_df(berlin_labels,berlin_times,berlin_weights,berlin_weight_classes)
amsterdam_df=to_df(amsterdam_labels,amsterdam_times,amsterdam_weights,amsterdam_weight_classes)

houston_df.to_csv('houston.csv',index=False)
berlin_df.to_csv('berlin.csv',index=False)
amsterdam_df.to_csv('amsterdam.csv',index=False)

# Australia

In [133]:
Australia_info=pd.read_csv('Hours_of_Australia.csv')
flat = lambda na:[v for a in na for v in a] #flattens a list of lists into just a list

raw_sts=Australia_info[['video']].to_numpy().astype('str')
sep_inds=[i for i in range(len(raw_sts)) if raw_sts[i][0]!='nan' and raw_sts[i][0][0]!='h']+[len(raw_sts)] #a!='nan'

Australia_Times=[flat(Australia_info[['Labeled Time(s)']][sep_inds[i]:sep_inds[i+1]].to_numpy().astype('int')) for i in range(len(sep_inds)-1)]
Australia_vid_lengths=flat(Australia_info[['video lengths(s)']][:36].to_numpy().astype('int')) #-1 just incase the videos are not actually as long as they say
for i in range(len(Australia_Times)):
    Australia_Times[i].append(Australia_vid_lengths[i]) #add the video length to the end of each times list
Australia_Times=[[[tl[i],tl[i+1]] for i in range(len(tl)-1)] for tl in Australia_Times] #rewrite times as start and stop times instead

Australia_Labels=[flat(Australia_info[['Labels']][sep_inds[i]:sep_inds[i+1]].to_numpy().astype('str')) for i in range(len(sep_inds)-1)]
Australia_Labels=[strs_to_labels(lbls) for lbls in Australia_Labels]
Australia_Labels=[full_relabel(lbls) for lbls in Australia_Labels]

#dict made using unique_aus_labels=np.unique(flat(Australia_Labels)).tolist();unique_w_labels=np.unique(weight_labels).tolist();generate_rough_dict(unique_aus_labels,unique_w_labels)
aus_to_w_dict={"A319":"A319","A320":"A320","A321":"A321","A330":"A330","A340":"A340","A350":"A350","A380":"A380","A72":"A72","AN":"An","B717-200":"B717-200BGW","B737":"B737-300","B737-300":"B737-300","B737-300F":"B737-300","B737-400F":"B737-400","B737-700":"B737-700","B737-700BBJ":"B737-700","B737-800":"B737-800","B737-900":"B737-900","B737F":"B737-300","B747":"B747-8","B747-400":"B747-400","B747-400ER":"B747-400ER","B747-400F":"B747-400ER","B747-8F":"B747-8F","B747F":"B747-8F","B757-200":"B757-200","B757-200F":"B757-200","B757F":"B757-200","B767":"B767-300","B767-300":"B767-300","B767-300ER":"B767-300ER","B767-300ERF":"B767-300ER","B767-300F":"B767-300ER","B767-30F":"B767-300ER","B767F":"B777F","B777":"B777F","B777-200":"B777-200ER","B777-200ER":"B777-200ER","B777-200F":"B777-200ER","B777-200LR":"B777-200LR","B777-300":"B777-300","B777-300ER":"B777-300ER","B777F":"B777F","B787":"B787-9","B787-10":"B787-10","B787-8":"B787-8","B787-800":"B787-8","B787-9":"B787-9","B787=9":"B787-9","BAe-146-200QTF":"BAE-146","BAe-146-300QTF":"BAE-146","C-130J":"C-130J","C-17":"C-17","D8":"Dash-8-Q400","DC-3":"DC-7","Dash-8":"Dash-8-Q400","E170":"E170","E190":"E190","F100":"F100","G6000":"G6000","IL-76":"IL-76","KC-30":"KC-30","MD-11F":"McDonnell-Douglas","RAAFFalcon7x":"RAAFFalcon7x","RJ-100":"RJ-100","S340":"S340"}

Australia_weights=[weight_extractor(lbls,aus_to_w_dict)[0] for lbls in Australia_Labels]
Australia_weight_classes=[weight_extractor(lbls,aus_to_w_dict)[1] for lbls in Australia_Labels]

Australia_dfs=[to_df(Australia_Labels[i],Australia_Times[i],Australia_weights[i],Australia_weight_classes[i]) for i in range(len(Australia_Labels))]
for i in range(len(Australia_dfs)):
    Australia_dfs[i].to_csv('Australia_'+str(i)+'.csv',index=False)

In [129]:
[time[-1] for time in Australia_Times]

[[2000, 2146],
 [1783, 1977],
 [1908, 1264],
 [1673, 1252],
 [1217, 1359],
 [1206, 1293],
 [1465, 1518],
 [1640, 1707],
 [1544, 1632],
 [1206, 1249],
 [1278, 1347],
 [1232, 1307],
 [1406, 1469],
 [1340, 1404],
 [1279, 1366],
 [1856, 1915],
 [1562, 1636],
 [1454, 1508],
 [1257, 1348],
 [1218, 1275],
 [1241, 1294],
 [1242, 1308],
 [1199, 1264],
 [8522, 8572],
 [1756, 1822],
 [1213, 1273],
 [1105, 1182],
 [1140, 1205],
 [1143, 1202],
 [1308, 1365],
 [1251, 1310],
 [1592, 1668],
 [1169, 1252],
 [1117, 1204],
 [1214, 1355],
 [1162, 1225]]

# Etc

In [ ]:
#use this to get the list of youtube video links
#[a for l in Australia_info[['video']].to_numpy().astype('str') for a in l if a[0]=='h'] #a!='nan' 

In [19]:
# Use the below code to generate a_to_w_dict (needs to be manually adjusted for best fit)
# Use the below code to generate a_to_w_dict (needs to be manually adjusted for best fit)
unique_aus_labels=np.unique(flat(Australia_Labels)).tolist();unique_w_labels=np.unique(weight_labels).tolist();generate_rough_dict(unique_aus_labels,unique_w_labels)

In [10]:
wcs=[houston_weight_classes,berlin_weight_classes,amsterdam_weight_classes]
[np.where(np.array(wc)=='Small')[0] for wc in wcs]

[array([ 1,  2, 25]), array([], dtype=int64), array([], dtype=int64)]

In [197]:
lbs=[houston_labels,berlin_labels,amsterdam_labels]
all_classes=np.unique([a for lst in [np.unique(lb).tolist() for lb in lbs] for a in lst]).tolist()
print(len(all_classes),"total classes")
all_classes


43 total classes


['A220',
 'A300',
 'A310',
 'A318',
 'A319',
 'A320',
 'A320N',
 'A321',
 'A330',
 'A350',
 'A380',
 'B737',
 'B737-700',
 'B737-800',
 'B737-900',
 'B747-400F',
 'B747-8F',
 'B747F',
 'B767',
 'B767-300ER',
 'B767-300F',
 'B767F',
 'B777',
 'B777-200ER',
 'B777-300ER',
 'B777F',
 'B787-10',
 'B787-8',
 'B787-9',
 'CRJ-1000',
 'CRJ-1000EL',
 'CRJ-900ER',
 'CRJ-900LR',
 'CRJ1000',
 'CRJ200',
 'CRJ900',
 'Dash-8-Q400',
 'E145',
 'E175',
 'E190',
 'E195',
 'MD80',
 'tiny']